In [ ]:
!pip install transformers
!pip install rouge

     |████████████████████████████████| 3.8 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 67 kB 3.5 MB/s 
     |████████████████████████████████| 6.5 MB 35.4 MB/s 
     |████████████████████████████████| 895 kB 65.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import csv
import random
import os
import re

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import pickle

## Prepare data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir("/content/drive/My Drive/proverbs")

In [ ]:
# загружаем датафрейм с описаниями предметов и выкидываем нулевые строки
df = pd.read_csv('/content/drive/MyDrive/proverbs/poverbs.csv')
df = df.dropna()

In [ ]:
# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df['Poverb'], df['Tag'], test_size=0.1, random_state=42)

## Prepare dataset

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [ ]:
# загружаем токенизатор и модель rugpt3
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
model = model.to(device)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
# класс датасетов для модели, в котором данные токенизируются
class DescDataset(Dataset):
    
    def __init__(self, X, y, max_length=1024, 
                 tokenizer=tokenizer):
        super().__init__()

        X.reset_index(drop=True, inplace = True)
        y.reset_index(drop=True, inplace = True)

        self.tokenizer = tokenizer
        self.end_token = "<|endoftext|>"
        self.start_token = "<|startoftext|>"
        self.descriptions = []

        for i in range(y.shape[0]):
          self.descriptions.append(
                self.tokenizer(
                    f"{self.start_token}{y[i]}. {X[i][:max_length]}{self.end_token}"))
                
        
    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, item):
        return self.descriptions[item]

In [ ]:
dataset = DescDataset(X_train, y_train)
dataset_test = DescDataset(X_test, y_test)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

## Model training

In [ ]:
batch_size = 64
epochs = 8
learning_rate = 3e-5
warmup_steps = 100
max_seq_len = 400

In [ ]:
# метод обучения модели
def train(model, output_path = "/content/drive/My Drive/proverbs/"):

    model = model.to(device)
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    tmp_items_tens = None
    for epoch in range(epochs):
        proc_seq_count = 0
        sum_loss = 0.0
        for _, item in tqdm(enumerate(data_loader), total=len(data_loader)):

            # хотим запихнуть как можно больше токенизированных итемов в последовательность длины max_seq_len
            item_tens = torch.tensor(item['input_ids']).to(device)

            # пропускаем если он длиннее max_seq_len
            if item_tens.size()[0] > max_seq_len:
                continue
            
            # кладем во временный накопительный тензор первый элемент
            if not torch.is_tensor(tmp_items_tens):
                tmp_items_tens = item_tens
                continue
            else:
                # если новый элемент не помещается в накопительный тензор, то мы кладем его во временный
                # а продолжаем работать с заполненным
                if tmp_items_tens.size()[0] + item_tens.size()[0] > max_seq_len:
                    work_items_tens = tmp_items_tens
                    tmp_items_tens = item_tens
                else:
                    # иначе кладем в накопительный тензор
                    tmp_items_tens = torch.cat([tmp_items_tens, item_tens[1:]], dim=0)
                    continue

            # обучаем модель
            outputs = model(work_items_tens, labels=work_items_tens)
            loss, logits = outputs[:2]                        
            loss.backward()
            sum_loss += loss.detach().data
                          
            if proc_seq_count % batch_size == 0:  
                optimizer.step()
                #scheduler.step() 
                optimizer.zero_grad()
                model.zero_grad()
            proc_seq_count +=  1
        print( f"Epoch {epoch+1} | Train loss: {sum_loss}")

        # сохраняем чекпоинты
        if not os.path.exists(output_path):
            os.mkdir(output_path)
        torch.save(model.state_dict(), os.path.join(output_path, f"rugpt2_proverb_{epoch+1}.pt"))
train(model)
            

  0%|          | 0/22773 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

### Своя функция для генерации (лучше не использовать)

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=35,
    top_p=0.8,
    temperature=0.8,
    model_path = "trained_models",
    model_epoch = 8):

    # загружаем обученную ранее модель
    model.load_state_dict(torch.load(os.path.join(
        model_path, f"distilgpt2_ds_{model_epoch}.pt"), 
        map_location=torch.device(device)))

    output_file_path = f'generated_{model_epoch}.txt'
    if os.path.exists(output_file_path):
        os.remove(output_file_path)

    model.eval()
    model = model.to('cpu')
        
    generated_num = 0
    generated_list = []
    with torch.no_grad():
      
            for idx in range(entry_count):
            
                description_finished = False

                # достаем поданный на вход промт и генерируем текст необходимой длины
                cur_ids = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

                for i in range(entry_length):
                    outputs = model(cur_ids, labels=cur_ids)
                    loss, logits = outputs[:2]
                    # преподготавливаем logits - скоры для всех словарных токенов
                    logits = logits[:, -1, :] / temperature

                    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                    sorted_indices_to_remove = cumulative_probs > top_p
                    sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
                    sorted_indices_to_remove[:, 0] = 0

                    indices_to_remove = sorted_indices[sorted_indices_to_remove]
                    logits[:, indices_to_remove] = -float("Inf")
                    
                    # случайно выбираем токен
                    next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                    cur_ids = torch.cat((cur_ids, next_token), dim=1)
                    if next_token in tokenizer.encode('<|endoftext|>'):
                        description_finished = True

                    # если встретили завершающий токен, значит завершаем генерацию
                    # и кладем результат в generated_list
                    if description_finished:
                        
                        generated_num += 1
                        
                        output_list = list(cur_ids.squeeze().numpy())
                        output_text = tokenizer.decode(output_list)
                        generated_list.append(output_text)

                        with open(output_file_path, 'a') as f:
                            f.write(f"{output_text}\n")
                        break
                # если до конца текста необходимой длины генерация не завершилась,
                # завершаем принудительно
                if not description_finished:
                    output_list = list(cur_ids.squeeze().numpy())
                    output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                    generated_list.append(output_text)
    return generated_list
              

### Нормальная генерация

In [ ]:
# генерация предложений с помощью исходного distilgpt2
def generate(prompt_text, model, tokenizer, n_seqs=1, max_length=35, min_length = 10):
    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")

    output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_length=max_length+len(encoded_prompt),
        min_length=min_length+len(encoded_prompt),
        temperature=0.8,
        top_p=0.8,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=n_seqs)

    # детокенизируем получившиеся последовательности в строку
    generated_list = []
    for seq in output_sequences:
        seq = seq.tolist()
        text = tokenizer.decode(seq)
        decoded_prompt = tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)
        total_sequence = (prompt_text + text[len(decoded_prompt):])
        generated_list.append(total_sequence)
    return generated_list


# генерация текста для списка названий предметов
def text_generation(test_data, model, tokenizer, gen_func='torch', entry_count=1):
    generated_descriptions = []
    for i in range(len(test_data)):
        if gen_func == 'torch':
            prompt = test_data[i] + '. '
            x = generate(prompt, model, tokenizer, n_seqs=entry_count)
        else:
            prompt = f'<|startoftext|>' + test_data[i] + f':'
            x = generate(model, tokenizer, prompt, entry_count=entry_count)
        for j in range(0, entry_count):
            x[j] = x[j].replace(prompt, '')
            x[j] = x[j].replace('\n', ' ')
            x[j] = re.sub('[<|>]', '', x[j])
            x[j] =x[j][: x[j].find("endoftext")]
            x[j] =x[j][: x[j].find("startoftext")]
            result = x[j].replace(x[j].split('.')[-1], '')
            if len(result.split())<5:
                result = x[j].replace("," + x[j].split(',')[-1], '.')
            x[j] = result

        generated_descriptions.append(x)
    return [test_data, generated_descriptions]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

model_path = '/content/drive/My Drive/proverbs/'
model.load_state_dict(torch.load(os.path.join(
        model_path, f"rugpt2_proverb_8_dot.pt"), 
        map_location=torch.device(device)))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [ ]:
gens = text_generation(['Тоска'], model, tokenizer, entry_count = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(gens)

[['Тоска'], [[' Скука, да и та тоска: кто скучает, тот не знает радости.', ' И я с тоски по миру бегу.', ' Да и сама тоска не своя, что в душу лезет.', ' Скука, тоска да вши — не беда.', ' Скука, как от холода зима сушит кости.']]]


In [ ]:
gens2 = text_generation(['Делу - время, потехе - час'], model, tokenizer, entry_count = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(gens2)

[['Делу - время, потехе - час'], [[' Иному и на копейку не прожить: один в поле воин; другой под гору бежит — от беды спасае', ' Деньгам - люди и те рады, кто деньги дает.', ' Не говори: "Хочу", а говори: "Было бы дело".И так до вечера.', ' Иные за работу в ответе.  Один не умеет: и с ложкой каши просит. (И.', ' Кто с огнем не дружит, тот от смерти умирает. Хлеб.']]]


In [ ]:
gens3 = text_generation(['Делу - время, потехе - час'], model, tokenizer, entry_count = 5)
print(gens3)